In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf

from fastDamerauLevenshtein import damerauLevenshtein
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from tensorflow.keras import layers
from tqdm import trange

In [77]:
df = pd.read_csv('data/single_step_df_ints_2022-09-12_encoded.csv', header=0)

/tmp/ipykernel_14932/4015456590.py:1: DtypeWarning: Columns (59,60,61,62,66,67,68,69,73,74,75,76,87,88,89,90,94,95,96,97,101,102,103,104,108,109,110,111,115,116,117,118,122,123,124,125,129,130,131,132,136,137,138,139,143,144,145,146,150,151,152,153,164,165,166,167,178,179,180,181,185,186,187,188,192,193,194,195,199,200,201,202,213,214,215,216,220,221,222,223,227,228,229,230,234,235,236,237,241,242,243,244,248,249,250,251,255,256,257,258,262,263,264,265,269,270,271,272,276,277,278,279,283,284,285,286,290,291,292,293,297,298,299,300,304,305,306,307,311,312,313,314,318,319,320,321,325,326,327,328,332,333,334,335,339,340,341,342,346,347,348,349,353,354,355,356) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('data/single_step_df_ints_2022-09-12_encoded.csv', header=0)


In [78]:
df

,input,target,start_coords.x,start_coords.y,start_coords.z,R.already_seen,x.already_seen,s.already_seen,silverware_1.already_seen,V.already_seen,...,F.food_k,F.strong_k,F.mid_k,coordinates_X.x,coordinates_X.y,coordinates_X.z,X.containment,X.food_k,X.strong_k,X.mid_k
0,<start>,p,-0.451354,-0.413918,0.156247,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,p,o,0.513000,-0.531000,0.740000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,o,c,0.513000,-0.531000,0.740000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,c,g,0.513000,-0.531000,0.740000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,g,k,0.513000,-0.531000,0.740000,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2198,O,R,2.500000,4.500000,2.000000,0.0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2199,R,S,2.500000,4.500000,2.000000,1.0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2200,S,T,2.500000,4.500000,2.000000,1.0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2201,T,U,2.500000,4.500000,2.000000,1.0,0.0,0.0,1.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [79]:
float_cols = df.select_dtypes(include=['float64']).columns
df.loc[:, float_cols] = df.loc[:, float_cols].fillna(-99)

In [80]:
cols = df.select_dtypes(include=['object'])
for col in cols.columns.values:
    df[col] = df[col].fillna('')

In [81]:
df['target'].value_counts()
df[:1437]['target'].value_counts()
#df[:1437]['target'].loc[df[:1437]['target'] == 'i'].index[0]
#df_new[1436:]['target'].value_counts()

p    188
k    162
g    160
f    157
c    148
s    134
b     78
w     74
o     73
a     72
r     47
d     27
h     27
t     22
e     22
z     21
m     13
n      9
x      2
i      1
Name: target, dtype: int64

In [82]:
#index_to_drop = df[1436:]['target'].loc[df[1436:]['target'] == 'y'].index[0]
#index_to_drop2 = df[1436:]['target'].loc[df[1436:]['target'] == 'h'].index[0]
index_to_drop = df['target'].loc[df['target'] == 'i'].index[0]
index_to_drop

727

In [12]:
index_to_drop2 = df_new[:1437]['target'].loc[df_new[:1437]['target'] == 'i'].index[0]
index_to_drop2

727

In [83]:
nr_of_sequences = len(df[df['input'] == '<start>'])

In [84]:
# remove row with target that only occurs once

df_new = df.drop(index=[index_to_drop], axis=0)
#df_new = df.drop(index=[index_to_drop2], axis=0)
df_new.reset_index(inplace=True, drop=True)

In [85]:
# generate stratified split

split = StratifiedShuffleSplit(n_splits=10, test_size=0.3, random_state=42)

for train_index, test_index in split.split(df_new[:1436], df_new[:1436]['target']):
#for train_index, test_index in split.split(df_new, df_new['target']):
#for train_index, test_index in split.split(df_new[1436:], df_new[1436:]['target']):
    strat_train = df_new.loc[train_index]
    strat_test_val = df_new.loc[test_index]

In [86]:
# split test_val into test and val

split_test_val = len(strat_test_val)//2

strat_test = strat_test_val[:split_test_val]
strat_val = strat_test_val[split_test_val:]

In [87]:
# generate list of labels to pass to MultiLabelBinarizer so there's the same number of
# classes for all datasets

labels = df_new['target'].unique()

In [88]:
len(labels)

42

In [94]:
def create_dataset(dataframe, labels, shuffle=True, batch_size=32):
    df = dataframe.copy()
    labels_ds = df.pop('target')
    encoder = MultiLabelBinarizer(classes=labels)
    encoded_labels = encoder.fit_transform(labels_ds)
    
    df = {key: value[:, tf.newaxis] for key, value in df.items()}
    dataset = tf.data.Dataset.from_tensor_slices((dict(df), encoded_labels))
    if shuffle:
        dataset = dataset.shuffle(buffer_size=len(dataframe))
    
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(batch_size)
    
    return dataset

In [71]:
batch_size = 128

In [95]:
train_ds = create_dataset(strat_train, labels, batch_size=batch_size)
val_ds = create_dataset(strat_val, labels, shuffle=False, batch_size=batch_size)
test_ds = create_dataset(strat_test, labels, shuffle=False, batch_size=batch_size)

/tmp/ipykernel_14932/1510453973.py:7: FutureWarning: Support for multi-dimensional indexing (e.g. `obj[:, None]`) is deprecated and will be removed in a future version.  Convert to a numpy array before indexing instead.
  df = {key: value[:, tf.newaxis] for key, value in df.items()}


ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type bool).

In [ ]:
def get_normalization_layer(name, dataset):
  # Create a Normalization layer for the feature.
    normalizer = layers.Normalization(axis=None)

  # Prepare a Dataset that only yields the feature.
    feature_ds = dataset.map(lambda x, y: x[name])

  # Learn the statistics of the data.
    normalizer.adapt(feature_ds)
    
    return normalizer

In [ ]:
def get_category_encoding_layer(name, dataset, dtype, max_tokens=None):
    if dtype == 'string':
        index = layers.StringLookup(max_tokens=max_tokens)
    else:
        index = layers.IntegerLookup(max_tokens=max_tokens)
        
    # prepare tf.data.Dataset that only yields the feature    
    feature_ds = dataset.map(lambda x, y: x[name])
    
    # learn set of possible values and assign fixed int index
    index.adapt(feature_ds)
    
    # encode int indices
    encoder = layers.CategoryEncoding(num_tokens=index.vocabulary_size())
    
    # apply multi-hot encoding to indices
    # lambda function captures the layer to include them in Keras functional models later
    return lambda feature: encoder(index(feature))

In [ ]:
def create_input_data(dataframe):
    all_inputs = []
    encoded_features = []
    
    for header in dataframe.columns:
        # numerical features
        if 'coord' in header or 'already' in header:
            numeric_col = tf.keras.Input(shape=(1,), name=header)
            normalization_layer = get_normalization_layer(header, train_ds)
            encoded_numeric_col = normalization_layer(numeric_col)
            all_inputs.append(numeric_col)
            encoded_features.append(encoded_numeric_col)
        
        # categorical features
        elif 'containment' in header or 'food' in header or 'mid' in header or \
        'strong' in header:
            categorical_col = tf.keras.Input(shape=(1,), name=header, dtype='int64')
            encoding_layer = get_category_encoding_layer(name=header,
                                                        dataset=train_ds,
                                                        dtype='int64')
            encoded_categorical_col = encoding_layer(categorical_col)
            all_inputs.append(categorical_col)
            encoded_features.append(encoded_categorical_col)
            
        elif header == 'input':
            categorical_col = tf.keras.Input(shape=(1,), name='input', dtype='string')
            encoding_layer = get_category_encoding_layer(name='input',
                                                        dataset=train_ds,
                                                        dtype='string')
            encoded_categorical_col = encoding_layer(categorical_col)
            all_inputs.append(categorical_col)
            encoded_features.append(encoded_categorical_col)
            
    return all_inputs, encoded_features

In [ ]:
all_inputs, encoded_features = create_input_data(df)

In [40]:
callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=5)

In [41]:
# use model from tutorial with dense layers

all_features = tf.keras.layers.concatenate(encoded_features)
x = tf.keras.layers.Dense(512, activation="relu")(all_features)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(256, activation="relu")(all_features)
x = tf.keras.layers.Dropout(0.5)(x)
output = tf.keras.layers.Dense(len(labels))(x)

model = tf.keras.Model(all_inputs, output)

In [24]:
model.compile(optimizer='adam',
             loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
             metrics=["accuracy"])

In [81]:
#tf.keras.utils.plot_model(model, show_shapes=True, rankdir='LR')

In [83]:
model.fit(train_ds, epochs=100, validation_data=val_ds, callbacks=[callback])

Epoch 1/100
4/4 [==============================] - 0s 46ms/step - loss: 1.2524 - accuracy: 0.5753 - val_loss: 0.8404 - val_accuracy: 0.7500
Epoch 2/100
4/4 [==============================] - 0s 46ms/step - loss: 1.1802 - accuracy: 0.5775 - val_loss: 0.8513 - val_accuracy: 0.7604
Epoch 3/100
4/4 [==============================] - 0s 46ms/step - loss: 1.1627 - accuracy: 0.5933 - val_loss: 0.8720 - val_accuracy: 0.7604
Epoch 4/100
4/4 [==============================] - 0s 44ms/step - loss: 1.1465 - accuracy: 0.6000 - val_loss: 0.8834 - val_accuracy: 0.7500
Epoch 5/100
4/4 [==============================] - 0s 46ms/step - loss: 1.1316 - accuracy: 0.5798 - val_loss: 0.8862 - val_accuracy: 0.7500
Epoch 6/100
4/4 [==============================] - 0s 47ms/step - loss: 1.0938 - accuracy: 0.6270 - val_loss: 0.8808 - val_accuracy: 0.7292
Epoch 7/100
4/4 [==============================] - 0s 46ms/step - loss: 1.1342 - accuracy: 0.5865 - val_loss: 0.8724 - val_accuracy: 0.7292
Epoch 8/100
4/4 [===

In [84]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

1/1 [==============================] - 0s 69ms/step - loss: 1.0930 - accuracy: 0.6526
Accuracy 0.6526315808296204


In [50]:
model.save('models/next_obj_classifier_tablesetting_2022-06-07')

2022-06-07 10:03:41.886830: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: models/next_obj_classifier_tablesetting_2022-06-07/assets


INFO:tensorflow:Assets written to: models/next_obj_classifier_tablesetting_2022-06-07/assets


In [3]:
reloaded_model = tf.keras.models.load_model('models/next_obj_classifier_tablesetting_2022-06-07/')

2022-06-29 13:44:36.463151: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-29 13:44:36.525754: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-29 13:44:36.526018: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:939] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-06-29 13:44:36.526328: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropri

In [ ]:
# test prediction for one sample

In [27]:
sample = df.loc[0].drop('target').to_dict()

In [28]:
input_dict = {name: tf.convert_to_tensor([value]) for name, value in sample.items()}

In [29]:
prediction = reloaded_model.predict(input_dict)
prediction = tf.nn.sigmoid(prediction[0])

In [30]:
# get label for prediction

pred_label = labels[np.argmax(prediction)]
pred_label

'p'

In [4]:
def get_prequential_error(dataframe, model, labels, nr_of_sequences):
    errors = [[] for seq in range(0, nr_of_sequences + 1)]
    predictions = [[] for seq in range(0, nr_of_sequences)]
    start_token_count = 0
    sequence_nr = 0
    
    for row in trange(dataframe.index[0], dataframe.index[-1]): 
        observed_target = dataframe.loc[row, 'target']
        sample = dataframe.loc[row].drop('target').to_dict()
        input_dict = {name: tf.convert_to_tensor([value]) for name, value in 
                          sample.items()}
        predicted_target = model.predict(input_dict)
        predicted_target = tf.nn.sigmoid(predicted_target[0])
            
        pred_label = labels[np.argmax(predicted_target)]
        error = 1 - damerauLevenshtein(pred_label, observed_target)
        errors[sequence_nr].append(error)
        predictions[sequence_nr].append(pred_label)
        
        if row != 0 and dataframe.loc[row, 'input'] == '<start>':
            start_token_count += 1
        
        if start_token_count > 0:
            sequence_nr += 1
            start_token_count = 0
            
    return errors, predictions

In [53]:
df[1438:].index[-1]

2073

In [15]:
errors, predictions = get_prequential_error(df, reloaded_model, labels, nr_of_sequences)

100%|████████████████████████████████████████████████████████████| 2073/2073 [11:49<00:00,  2.92it/s]


In [16]:
summed_error = [sum(error) for error in errors[:-1]]

In [17]:
np.median(summed_error)

3.0

In [57]:
with open('results/nn_spatialinfo_prequential_summed_fitted_on_ts_results_all_2022-06-07.txt', 'w') as file:
    file.write(str(summed_error))

In [58]:
np.mean(summed_error)

3.466019417475728

In [34]:
results = pd.DataFrame()

In [90]:
results['fitted_on_cooking_all'] = summed_error

In [24]:
df_ids = pd.read_csv('../../opportunistic_planning/all_lowest_error/all_task_environments_list_2022-06-02_unique_sequences.csv')

In [91]:
results

,fitted_on_ts_all,fitted_on_all_all,ID,fitted_on_cooking_all
0,1.0,1.0,a1,2.0
1,4.0,3.0,a3,2.0
2,3.0,3.0,a5,3.0
3,8.0,8.0,a11,8.0
4,2.0,0.0,a13,1.0
...,...,...,...,...
304,6.0,5.0,s37-d21,5.0
305,3.0,3.0,s37-d25,3.0
306,4.0,3.0,s37-d29,3.0
307,8.0,6.0,s37-d39,6.0


In [93]:
results.to_csv('results/nn_spatialinfo_prequential_fitting_variants_2022-06-23.csv', header=True, index=False)

In [23]:
prediction_df = pd.read_csv('results/predictions_comparison_2022-06-29.csv', header=0)

In [19]:
predictions = [''.join(lst) for lst in predictions]

In [20]:
prediction_df['nn_fitted_ts'] = predictions

In [25]:
prediction_df['ID'] = df_ids['ID']

In [26]:
prediction_df.to_csv('results/predictions_comparison_2022-06-29.csv', header=True, index=False)